In [4]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

# I] Import

In [1]:
import pandas

# II] ATG7 isoform expression

In [5]:
%%time
# DL data ATG7 isoform expression
path = output_file_directory + "/ATG7/TCGA_GTEX/dataframes_for_input/df_ATG7_NormPrim.csv"
df_ATG7_NormPrim= pandas.read_csv(path, sep = ",")

#remove Survival information
df_ATG7_NormPrim.drop(df_ATG7_NormPrim.columns[[3, 4]], axis = 1, inplace = True)

print(df_ATG7_NormPrim.shape)
df_ATG7_NormPrim.head(2)

(16614, 20)
Wall time: 87.7 ms


,sample,Sample_Type,Primary_Site,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,log2_p1_ATG7_total,log2_p1_ATG7_1,log2_p1_ATG7_2,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,GTEX-ZTTD-0326-SM-57WFW,Normal Tissue,Muscle,5.042,3.860,2.390,0.2642,32.944283,14.519306,5.240574,1.199970,5.085097,3.955992,2.641679,20.95985,69.271996,25.002916,5.725088,0.440723,0.159074
1,GTEX-PX3G-1626-SM-2S1PT,Normal Tissue,Muscle,4.252,2.322,1.956,1.1320,19.052710,4.999249,3.878848,2.190624,4.325725,2.584782,2.286540,11.06872,45.165557,35.043325,19.791118,0.262390,0.203585


In [8]:
#split to have Normal (GTEX) and Primary (TCGA)
df_ATG7_Normal = df_ATG7_NormPrim[df_ATG7_NormPrim['sample'].str.startswith('GTEX')]
print(len(df_ATG7_Normal))
df_ATG7_Primary = df_ATG7_NormPrim[df_ATG7_NormPrim['sample'].str.startswith('TCGA')]
print(len(df_ATG7_Primary))

7429
9185


## A) ATG7 isoforms + Whole gene expression (ensembl)

In [12]:
%%time
# DL data gene expression for Normal Tissue and Primary Tumors
path = output_file_directory + "dataframes_for_input/liver/"

ensembl_normal_liver_protcoding_expr= pandas.read_csv(path+ 'ensembl_normal_liver_protcoding_expr.tsv', sep = "\t")
print(ensembl_normal_liver_protcoding_expr.shape)

directory = "ensembl_primary_liver_protcoding_expr.tsv"
ensembl_primary_liver_protcoding_expr= pandas.read_csv(path+'ensembl_primary_liver_protcoding_expr.tsv' , sep = "\t")
print(ensembl_primary_liver_protcoding_expr.shape)
ensembl_primary_liver_protcoding_expr.head()

(110, 19383)
(369, 19383)
Wall time: 7.64 s


,sample,ENSG00000198888,ENSG00000198763,ENSG00000198804,ENSG00000198712,ENSG00000228253,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,...,ENSG00000160678,ENSG00000160679,ENSG00000143553,ENSG00000214193,ENSG00000196182,ENSG00000181817,ENSG00000116885,ENSG00000116898,ENSG00000119535,ENSG00000142694
0,TCGA-2V-A95S-01,10.7384,11.9699,11.7672,13.1304,11.6930,11.8651,12.1800,11.8556,9.9512,...,2.4675,6.0665,5.4209,-0.2159,4.0091,5.3352,-0.3940,6.4346,3.2220,2.7847
1,TCGA-2Y-A9GS-01,12.3783,12.1026,13.7137,13.2767,13.0898,13.1596,13.9598,12.4285,12.2669,...,2.1606,5.6996,5.1157,0.0014,4.2220,5.1962,-0.5125,6.6128,1.9931,3.8340
2,TCGA-2Y-A9GT-01,12.0660,12.0678,13.7570,13.5460,13.8357,13.6818,13.4631,12.0764,12.7607,...,1.5266,4.3702,4.1111,-0.8599,4.0375,4.5129,-0.5973,5.6542,1.3956,2.3366
3,TCGA-2Y-A9GU-01,12.6688,12.9656,14.8855,14.5545,14.2781,14.1697,14.4867,13.2030,13.6372,...,0.9115,5.0674,4.8213,0.2029,3.1556,4.6668,0.3796,6.4754,-0.8084,1.7617
4,TCGA-2Y-A9GV-01,13.1180,13.5375,14.1719,13.8771,13.9111,13.6207,13.8645,13.7041,13.2222,...,0.5370,4.4686,4.3140,-1.1172,3.9910,4.5280,0.4761,5.8810,1.1447,2.7183


In [14]:
#Preparation of the global dataframe with ATG7 isoform expression AND whole gene expression
data = [[ensembl_normal_liver_protcoding_expr, df_ATG7_Normal], [ensembl_primary_liver_protcoding_expr, df_ATG7_Primary]]

df_final = []

for df in data:
    #Merge the two df to have ATG7 isoform expression + whole gene expression 
  df_merged = pandas.merge(df[1], df[0], on= 'sample')

  #delete the sample column and select column
  df_merged = df_merged.drop("sample", 1)
  df_merged = df_merged.drop(df_merged.columns[5:19], axis=1)
  
  #save in list
  df_final.append(df_merged)

ensembl_normal_liver_protcoding_atg7_expr = df_final[0]
ensembl_primary_liver_protcoding_atg7_expr = df_final[1]

print(ensembl_normal_liver_protcoding_atg7_expr.shape)
print(ensembl_primary_liver_protcoding_atg7_expr.shape)
ensembl_normal_liver_protcoding_atg7_expr.head()

(110, 19387)
(369, 19387)


,Sample_Type,Primary_Site,ATG7,ATG7_1,ATG7_2,ENSG00000198888,ENSG00000198763,ENSG00000198804,ENSG00000198712,ENSG00000228253,...,ENSG00000160678,ENSG00000160679,ENSG00000143553,ENSG00000214193,ENSG00000196182,ENSG00000181817,ENSG00000116885,ENSG00000116898,ENSG00000119535,ENSG00000142694
0,Normal Tissue,Liver,4.146,2.6020,2.2110,12.1227,12.4690,12.8944,12.8383,13.4944,...,0.9038,5.9028,4.9477,1.9073,5.9903,5.7578,1.4911,6.6100,6.8698,5.0854
1,Normal Tissue,Liver,3.453,0.5069,2.2080,13.0273,13.4080,13.5398,13.9006,14.5052,...,0.6699,5.5808,4.1539,0.0014,5.3548,5.0211,1.0151,6.3555,3.3105,2.2082
2,Normal Tissue,Liver,3.313,2.1080,-0.8339,12.6203,13.3898,14.1408,13.7905,14.2730,...,0.1648,5.3288,4.6826,0.9115,5.1098,5.5245,0.4233,6.2159,3.1228,3.6782
3,Normal Tissue,Liver,3.259,2.7010,-1.4310,8.9743,11.2312,10.6905,12.2497,12.0025,...,1.7229,5.9892,5.4738,3.0550,4.8929,4.9031,3.2766,6.3815,3.3278,6.2589
4,Normal Tissue,Liver,3.177,1.5560,0.7233,12.3994,12.8692,13.7866,14.5786,14.9837,...,1.2815,4.4686,4.8420,-0.2328,4.4719,5.4019,0.4865,6.7093,4.9556,4.2803


In [15]:
#save to csv
path = 'dataframes_for_input/liver/'
ensembl_normal_liver_protcoding_atg7_expr.to_csv(output_file_directory+path+'ensembl_normal_liver_atg7_proteinGE.tsv', sep = "\t", index=False)
ensembl_primary_liver_protcoding_atg7_expr.to_csv(output_file_directory+path+'ensembl_primary_liver_atg7_proteinGE.tsv', sep = "\t", index=False)

In [ ]:
##SYMBOL NOT UPDATED

## B) ATG7 isoforms + Whole gene expression (symbol)

In [14]:
%%time
# DL data gene expression for Normal Tissue and Primary Tumors
input_file_directory_gene_expr = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 Python Analysis/gene_expr_preparation/'

directory = input_file_directory_gene_expr + "symbol_normal_liver_protcoding_expr.tsv"
symbol_normal_liver_protcoding_expr= pandas.read_csv(directory, sep = "\t")
print(symbol_normal_liver_protcoding_expr.shape)

directory = input_file_directory_gene_expr + "symbol_primary_liver_protcoding_expr.tsv"
symbol_primary_liver_protcoding= pandas.read_csv(directory, sep = "\t")
print(symbol_primary_liver_protcoding.shape)
symbol_primary_liver_protcoding.head(2)

(7429, 19190)
(9185, 19190)
Wall time: 3min 31s


,sample,MT-ND1,MT-ND2,MT-CO1,MT-CO2,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,...,S100A1,CHTOP,SNAPIN,SH3D21,STK40,LSM10,OSCP1,MRPS15,CSF3R,EVA1B
0,TCGA-02-0047-01,12.9202,13.1220,14.0136,13.5251,13.8439,13.7777,13.4037,12.8123,13.4187,...,6.4741,5.8017,6.4975,2.1509,5.2445,6.0507,3.8915,5.5389,4.2936,4.6685
1,TCGA-02-0055-01,10.3972,10.7774,12.3945,12.3893,11.7280,11.9408,12.2022,11.0142,11.0221,...,5.9969,5.6888,6.0633,1.9822,4.3299,6.4992,3.7582,6.7092,4.5964,5.5145


In [15]:
#Preparation of the global dataframe with ATG7 isoform expression AND whole gene expression
data = [[symbol_normal_liver_protcoding, df_ATG7_Normal], [symbol_primary_liver_protcoding, df_ATG7_Primary]]

df_final = []

for df in data:
    #Merge the two df to have ATG7 isoform expression + whole gene expression 
  df_merged = pandas.merge(df[1], df[0], on= 'sample')

  #delete the sample column and select column
  df_merged = df_merged.drop("sample", 1)
  df_merged = df_merged.drop(df_merged.columns[5:19], axis=1)
  
  #save in list
  df_final.append(df_merged)

symbol_normal_liver_protcoding_atg7_expr = df_final[0]
symbol_primary_liver_protcoding_atg7_expr = df_final[1]

print(symbol_normal_liver_protcoding_atg7_expr.shape)
print(symbol_primary_liver_protcoding_atg7_expr.shape)
symbol_normal_liver_protcoding_atg7_expr.head()

(7429, 19191)
(9185, 19191)


,Sample_Type,Primary_Site,ATG7_x,ATG7_1,ATG7_2,MT-CO2,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,...,S100A1,CHTOP,SNAPIN,SH3D21,STK40,LSM10,OSCP1,MRPS15,CSF3R,EVA1B
0,Normal Tissue,Muscle,5.042,3.860,2.390,14.4453,15.0971,14.4720,14.7683,13.8143,...,2.7868,5.6494,5.0193,2.6940,4.5274,5.0998,2.3423,6.0790,6.4861,4.9700
1,Normal Tissue,Muscle,4.252,2.322,1.956,14.3855,15.1867,14.8991,14.4038,13.1051,...,9.3501,6.1104,6.0570,1.1770,6.8792,6.4325,2.1013,6.7035,1.1706,1.3167
2,Normal Tissue,Muscle,3.909,2.602,1.345,13.8211,14.8161,14.1107,14.7035,13.2776,...,9.0646,5.9750,5.1173,1.8683,4.6136,5.0090,2.0604,5.7848,1.7489,4.6514
3,Normal Tissue,Muscle,3.854,2.568,1.428,14.0622,15.5897,14.8774,14.5459,13.7788,...,9.2252,5.4941,5.5880,-0.4521,5.6631,6.3797,2.5683,7.3214,1.6140,1.5563
4,Normal Tissue,Muscle,3.822,2.164,1.293,13.4082,14.0564,13.7832,13.5560,13.2383,...,8.9779,6.1502,5.1330,2.0289,6.8202,6.1789,1.8524,6.7685,2.8838,1.8323


In [30]:
#Why ATG7_x ?
verif = symbol_normal_liver_protcoding_atg7_expr[symbol_normal_liver_protcoding_atg7_expr.columns[pandas.Series(symbol_normal_liver_protcoding_atg7_expr.columns).str.startswith('ATG7')]]
print(verif.head(2))

   ATG7_x  ATG7_1  ATG7_2  ATG7_y
0   5.042   3.860   2.390  5.0418
1   4.252   2.322   1.956  4.2518


In [33]:
data = [symbol_normal_liver_protcoding_atg7_expr, symbol_primary_liver_protcoding_atg7_expr]

for df in data:
    df.drop('ATG7_y', axis = 1, inplace=True) 
    df.rename(columns = {'ATG7_x' : 'ATG7'}, inplace=True)
    verif = df[df.columns[pandas.Series(df.columns).str.startswith('ATG7')]]
    print(verif.head(2))

    ATG7  ATG7_1  ATG7_2
0  4.824   3.239   1.903
1  4.790   4.056   1.151


In [35]:
#save to csv
# symbol_normal_liver_protcoding_atg7_expr.to_csv('symbol_normal_liver_atg7_ProteinGE.tsv', sep = "\t", index=False)
# symbol_primary_liver_protcoding_atg7_expr.to_csv('symbol_primary_liver_atg7_ProteinGE.tsv', sep = "\t", index=False)